# Final Project Submission

Please fill out:
* Student name: Joseph Husney
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: James Irving
* Blog post URL:


## Introduction
In this project, we will analyze data from themoviedb3 api and use that information to shed some light on which movies are more profitable than other. This will help microsoft decide which movies to get into

In [ ]:
# Import modules needed for project
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
from tqdm import tqdm

In [ ]:
# Allowing pandas to display all info
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

#### Retrieve id's to get details about
Before analyzing the data, it must first be retrieved. Inside the package given to us, there is a file that has all the movie id's. These Id's will tell the api which movies to get data about. This code will get all the id's and put them into a list

In [ ]:
df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
# print(len(df))
# print(df.columns)
movie_ids = []
for index, rows in df.iterrows():
    if int(df['release_date'][0].split('-')[0]) > 2000:
        my_list = [rows.id]
        movie_ids.append(my_list)
print(movie_ids[:20])

In [ ]:
len(movie_ids)

Function to read key from file in path

In [1]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

## Retrieve all data from moviedb api
In order to retrieve all the data from the api, you need to go to the website and sign up for a unique api key. Using that and other parameters such as language, one can link into their api to get the details for the movie id's we have retrieved. 
Using a loop to change the url for different movie each time

In [ ]:
# Getting so far info about 100 movies based on 100 id's that I got from dataset I was given
# There are 26 thousand movies but it takes very long
movie_info = []
keys = get_keys("/Users/joey/.secret/the_movie_db_api.json")
api_key = keys['api_key']
headers = {
        'Authorization': 'Bearer {}'.format(api_key)
    }

url_params = {
                'language':'en-US'
            }

for id_ in tqdm(movie_ids[:1000]):
    url = f"https://api.themoviedb.org/3/movie/{id_[0]}"
    response = requests.get(url, headers=headers, params=url_params)
    movie_info.append(response.json())

Once we have retrieved all the details for these movies, we will analyze it through the lens of a pandas dataframe

In [ ]:
df = pd.DataFrame.from_dict(movie_info)
df.head()

## Data Cleaning
Upon inspecting the data, it became clear that there are many movies missing certain important data points. For instance, some movies don't have genre data. Others have a budget and/or revenue of zero. Here we will clean up the data by deleting those rows from the dataframe.

#### Get rid of rows without genre data

In [ ]:
df = df[~df['genres'].isna()]

#### Get rid of rows with budget or revenue of zero

In [ ]:

indexNames = df[ df['budget'] == 0 ].index
df.drop(indexNames , inplace=True)


indexNames = df[ df['revenue'] == 0 ].index
df.drop(indexNames , inplace=True)

## Question 1: What is the average gross profit margin for each movie genre?

In [ ]:
def seperate_genres(genre_list):
    genres = []
    for genre in genre_list:
            genres.append(genre['name'])
    return genres

In [ ]:
len(df['genres'])

In [ ]:
df['genres'].isna().sum()

In [ ]:
df['genre_list'] = df['genres'].map(seperate_genres)

In [ ]:
genre_df_explode = df.explode('genre_list')

In [ ]:
genre_df_explode['profit'] = genre_df_explode['revenue'] - genre_df_explode['budget']

In [ ]:
genre_df_explode['profit_margin'] = (genre_df_explode['profit'] / genre_df_explode['revenue'])*100

In [ ]:
genre_df_explode.head(1000)

In [ ]:
sns.barplot(y= 'genre_list', x='profit_margin', data=genre_df_explode, ci=68)